In [ ]:
"""
Read an image & Brand Info -> convert to base64 >> simple python code

agentic starts here:
1. send to openai for image detailed description ->
2. send description to creative model for creative ideas prompts -> 
3. send image, brand info, creative ideas prompts to creative model for design ->
4. covert image back from base64 -> 
5. Compress image to < 1MB & 24 Bit depth & Resize to 1080x1920 for TIKTOK ->
6. save image & save design prompt.
"""

In [31]:
from langchain_core.pydantic_v1 import BaseModel, Field
from openai import OpenAI
import base64
from langchain_core.runnables import RunnableLambda, RunnableSequence
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Read image and brand profile

In [20]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

## 1

In [28]:
IMAGE_DESCRIPTION_PROMPT_TEMPLATE = """ 
Describe the image in detail, focus on the main subject in the image usually in the center, 
extract all brand info like brand name and slogan if applicable. Make sure to include all details of the image.
"""
client = OpenAI(api_key=OPENAI_API_KEY)

def describe_image(base64_image: str) -> str:
    response = client.responses.create(
        model="gpt-4.1",
        input=[
            {
                "role": "user",
                "content": [
                    { "type": "input_text", "text": IMAGE_DESCRIPTION_PROMPT_TEMPLATE },
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{base64_image}",
                    },
                ],
            }
        ],
    )

    return response.output_text
image_description_runnable = RunnableLambda(describe_image)

## 2

In [32]:
creative_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# Give the LLM a system instruction for creativity
creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative AI for marketing."),
    ("user", 
     "Given an image description, generate 3 imaginative image ideas for image-to-image model, the ideas should be for marketing and online presence."
     "Here is the image description:\n\n{description}")
])
# Combine prompt + model into a runnable
creative_runnable = creative_prompt | creative_llm


chain = image_description_runnable | creative_runnable

In [33]:
# Getting the Base64 string
image_path = "images/img.jpg"
base64_image = encode_image(image_path)

# Run chain
result = chain.invoke(base64_image)
print(result.content)

Here are three imaginative image ideas for marketing and online presence based on the provided image description:

1. **Cultural Feast Experience**:
   Create an inviting outdoor setting at sunset, showcasing the mouthwatering platter as the centerpiece of a communal feast. Surround the platter with colorful decorative cushions and traditional woven textiles. Add rustic wooden tables adorned with flickering lanterns and small floral arrangements. Include diverse groups of friends and family joyfully sharing the meal, laughter echoing. Captions can play with themes of togetherness, heritage, and the joy of sharing traditional food. Utilize hashtags like #CulturalFeast and #FoodStories to connect with audiences seeking authentic experiences.

2. **Step-by-Step Culinary Journey**:
   Design a multi-image carousel that takes viewers through the cooking process, from raw ingredients to the finished platter. Start with vibrant images of spices, herbs, and fresh ingredients being artfully arr

In [34]:
from openai import OpenAI
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# --- Step 1: Image -> Description ---
IMAGE_DESCRIPTION_PROMPT_TEMPLATE = """ 
Describe the image in detail, focus on the main subject in the image usually in the center, 
extract all brand info like brand name and slogan if applicable. Make sure to include all details of the image.
"""

client = OpenAI(api_key=OPENAI_API_KEY)

def describe_image(base64_image: str) -> dict:
    response = client.responses.create(
        model="gpt-4.1",  # vision-capable
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": IMAGE_DESCRIPTION_PROMPT_TEMPLATE},
                    {"type": "input_image", "image_url": f"data:image/jpeg;base64,{base64_image}"},
                ],
            }
        ],
    )
    # Return dict so we can pass both description + original image forward
    return {
        "description": response.output_text,
        "base64_image": base64_image
    }

image_description_runnable = RunnableLambda(describe_image)

# --- Step 2: Creative Ideas (use description + image) ---
creative_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative AI for marketing."),
    ("user", 
     "Given image description AND the actual image, generate 3 imaginative image ideas for image-to-image model, the ideas should be for marketing and online presence."
     "Here is the image description:\n\n{description}\n\n"),
    # attach the raw image again
    ("user", [
        {"type": "image_url", "image_url": {"url": "data:image/jpeg;base64,{base64_image}"}}
    ])
])

# Combine prompt + LLM
creative_runnable = creative_prompt | creative_llm

# --- Build full chain ---
chain = image_description_runnable | creative_runnable

# Example usage
image_path = "images/img.jpg"
base64_image = encode_image(image_path)

result = chain.invoke(base64_image)
print(result.content)


Here are three imaginative image ideas for marketing and online presence based on the provided image description:

1. **Cultural Feast Celebration**:
   - Create a vibrant scene depicting a communal dining table set outdoors, adorned with intricate Middle Eastern decor. The focal point is the large tray of Kabsa, surrounded by family and friends enjoying the meal. Fresh salads, dips, and drinks are artfully arranged on the table, promoting a sense of community and celebration. Soft lighting during golden hour can enhance the ambiance.

2. **Chef's Special Showcase**:
   - Illustrate a close-up of a chef in traditional attire, skillfully garnishing the Kabsa with fresh herbs and lemon slices. Incorporate elements that highlight the cooking process, such as spices and fresh ingredients scattered around. The background can feature a glowing kitchen atmosphere, emphasizing authenticity and craftsmanship in traditional Middle Eastern cuisine.

3. **Flavor Journey with Pairings**:
   - Desig

In [42]:
import base64
import json
from openai import OpenAI
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import AIMessage

# --- Helper: encode image to base64 ---
def encode_image(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# --- OpenAI client ---
client = OpenAI(api_key=OPENAI_API_KEY)

# --- Step 1: Image -> Description ---
IMAGE_DESCRIPTION_PROMPT_TEMPLATE = """ 
Describe the image in detail, focus on the main subject in the image usually in the center, 
extract all brand info like brand name and slogan if applicable. Make sure to include all details of the image.
"""

def describe_image(base64_image: str) -> dict:
    response = client.responses.create(
        model="gpt-4.1",  # vision-capable
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": IMAGE_DESCRIPTION_PROMPT_TEMPLATE},
                    {"type": "input_image", "image_url": f"data:image/jpeg;base64,{base64_image}"},
                ],
            }
        ],
    )
    return {
        "description": response.output_text,
        "base64_image": base64_image
    }

image_description_runnable = RunnableLambda(describe_image)

# --- Step 2: Description + Image -> Creative Prompts ---
creative_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

creative_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative AI for marketing."),
    ("user", 
     "Here is the image description:\n\n{description}\n\n"
     "Based on this description AND the actual image, generate 3 imaginative "
     "**image generation prompts** to create new marketing visuals. "
     "Return only JSON with keys: prompt1, prompt2, prompt3. "
     "Do NOT include any extra text or explanations."),
    ("user", [
        {"type": "image_url", "image_url": {"url": "data:image/jpeg;base64,{base64_image}"}}
    ])
])


creative_runnable = creative_prompt | creative_llm
import re

def parse_json_safe(text: str) -> dict:
    # Extract {...} from the string
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("No JSON found in AI output")

# --- Step 3: Prompts -> Images using GPT-4.1 ---
def generate_images_from_prompts(ai_message: AIMessage) -> dict:
    prompts_json = parse_json_safe(ai_message.content)
    images = {}
    for key, prompt in prompts_json.items():
        resp = client.responses.create(
            model="gpt-4.1",
            input=[{"role": "user", "content": [{"type": "input_text", "text": prompt}]}],
        )
        # GPT-4.1 returns base64 string in resp.output[0].content[0].text
        b64_image = resp.output[0].content[0].text
        print(b64_image)
        print(type(b64_image))
        image_bytes = base64.b64decode(b64_image)
        
        # Save image to file
        file_path = f"{key}.png"
        with open(file_path, "wb") as f:
            f.write(image_bytes)
        
        images[key] = file_path
    return images

image_generation_runnable = RunnableLambda(generate_images_from_prompts)

# --- Full Chain: Image -> Description -> Creative Prompts -> Generated Images ---
chain = image_description_runnable | creative_runnable | image_generation_runnable

# --- Run the chain ---
image_path = "images/img.jpg"  # path to your input image
base64_image = encode_image(image_path)

result = chain.invoke(base64_image)
print(result)  # dict with {"prompt1": <image_url>, "prompt2": ..., "prompt3": ...}


Certainly! Here’s a detailed description of such an image:

A large, round metal tray brimming with *Kabsa*—golden rice crowned with succulent pieces of spiced chicken, roasted nuts, and raisins—takes center stage atop a red-and-white checkered picnic blanket spread over lush green grass. Around the tray, smaller bowls overflow with colorful salads: a fresh tabbouleh shimmering with chopped parsley and tomato; creamy hummus swirled with olive oil and paprika; and a bright cucumber-yogurt salad. Stacks of warm, pillowy pita and crusty bread sit ready for tearing and sharing.

Glasses and pitchers of cool drinks—perhaps mint lemonade and chilled water—sparkle in the sunlight. People, a diverse group of friends and family, are seated cross-legged and barefoot, their hands reaching toward the tray, passing platters, laughing, and enjoying each other’s company. Above, the sky is bright and clear, with sunlight dappled through a few leaves, completing the joyful, communal atmosphere of the p

ValueError: string argument should contain only ASCII characters